<a href="https://colab.research.google.com/github/oleksii-shcherbak/GoIT-num-prog-py-hw/blob/main/hw_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1

In [13]:
import gym
import numpy as np

# Create the FrozenLake environment
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True, render_mode="rgb_array")

def show_render(env):
    """
    Function to visualize the environment state
    """
    import matplotlib.pyplot as plt
    img = env.render()
    plt.imshow(img)
    plt.axis('off')
    plt.title('FrozenLake Environment')
    plt.show()

### Step 2

In [14]:
def compute_value_function(policy, gamma=1.0):
    """
    Compute the value function for a given policy using iterative policy evaluation.

    Parameters:
    policy: array of shape (n_states,) - policy to evaluate
    gamma: float - discount factor

    Returns:
    value_table: array of shape (n_states,) - value function for the given policy
    """
    # Initialize value table with zeros
    value_table = np.zeros(env.observation_space.n)

    # Set number of iterations and convergence threshold
    no_of_iterations = 100000
    threshold = 1e-10

    for i in range(no_of_iterations):
        # Copy current value table for comparison
        updated_value_table = np.copy(value_table)

        # For each state, compute value using the given policy
        for state in range(env.observation_space.n):
            # Get action from policy for current state
            action = int(policy[state])

            # Compute value for this state-action pair
            value_table[state] = sum([
                trans_prob * (reward_prob + gamma * updated_value_table[next_state])
                for trans_prob, next_state, reward_prob, _ in env.P[state][action]
            ])

        # Check for convergence
        if np.sum(np.fabs(updated_value_table - value_table)) <= threshold:
            print(f'Policy evaluation converged at iteration {i+1}')
            break

    return value_table

### Step 3

In [15]:
def policy_iteration(env, gamma=1.0):
    """
    Find the optimal policy using policy iteration algorithm.

    Parameters:
    env: gym environment
    gamma: float - discount factor

    Returns:
    optimal_policy: array of shape (n_states,) - optimal policy
    optimal_value_function: array of shape (n_states,) - optimal value function
    """
    # Initialize random policy (all zeros)
    old_policy = np.zeros(env.observation_space.n)
    no_of_iterations = 1000

    for i in range(no_of_iterations):
        # Policy Evaluation: compute value function for current policy
        new_value_function = compute_value_function(old_policy, gamma)

        # Policy Improvement: extract new policy from value function
        new_policy = extract_policy(new_value_function, gamma)

        # Check for convergence
        if np.array_equal(old_policy, new_policy):
            print(f'Policy iteration converged at step {i+1}')
            break

        # Update policy
        old_policy = new_policy

    return new_policy, new_value_function

def extract_policy(value_table, gamma=1.0):
    """
    Extract policy from value function.

    Parameters:
    value_table: array of shape (n_states,) - value function
    gamma: float - discount factor

    Returns:
    policy: array of shape (n_states,) - extracted policy
    """
    # Initialize policy with zeros
    policy = np.zeros(env.observation_space.n)

    for state in range(env.observation_space.n):
        # Initialize Q-table for current state
        Q_table = np.zeros(env.action_space.n)

        # Compute Q-value for each action in current state
        for action in range(env.action_space.n):
            for trans_prob, next_state, reward_prob, _ in env.P[state][action]:
                Q_table[action] += trans_prob * (reward_prob + gamma * value_table[next_state])

        # Select action with maximum Q-value
        policy[state] = np.argmax(Q_table)

    return policy

### Step 4

In [16]:
# Find optimal policy using policy iteration
print("Starting Policy Iteration...")
optimal_policy, optimal_value_function = policy_iteration(env, gamma=0.9)

print("\nOptimal Value Function:")
print(optimal_value_function.reshape(4, 4))

print("\nOptimal Policy:")
policy_grid = optimal_policy.reshape(4, 4)
print(policy_grid)

# Convert policy numbers to direction symbols for better visualization
action_symbols = {0: '←', 1: '↓', 2: '→', 3: '↑'}
print("\nOptimal Policy (with direction symbols):")
for i in range(4):
    row = ""
    for j in range(4):
        action = int(policy_grid[i, j])
        row += action_symbols[action] + " "
    print(row)

# Simple policy demonstration without extensive testing
print("\nDemonstrating the optimal policy with a few steps:")

# Reset environment manually
state = 0  # Start position
print(f"Starting at state: {state} (position [0,0])")

# Show first few optimal moves
print("\nOptimal actions from different states:")
for s in range(16):
    if s not in [5, 7, 11, 12, 15]:  # Skip hole and goal states
        action = int(optimal_policy[s])
        action_name = ['←', '↓', '→', '↑'][action]
        row, col = s // 4, s % 4
        print(f"State {s} (position [{row},{col}]): Action {action} ({action_name})")

# Show the environment layout
print("\nEnvironment Layout Reference:")
print("S = Start(0), F = Frozen, H = Hole, G = Goal(15)")
print("State numbers:")
print("[ 0  1  2  3 ]")
print("[ 4  H  6  H ]")
print("[ 8  9 10  H ]")
print("[ H 13 14  G ]")

print("\nPolicy Interpretation:")
print("The policy shows the optimal direction to move from each state")
print("to maximize the probability of reaching the goal (state 15)")
print("while avoiding holes (states 5, 7, 11, 12)")

# Simple manual trace of one path
print("\nExample path following the policy:")
current_state = 0
path = [current_state]
visited_states = set()

print(f"Start: State {current_state}")
for step in range(10):  # Limit to 10 steps to avoid infinite loops
    if current_state in visited_states:
        print("Loop detected - stopping trace")
        break
    visited_states.add(current_state)

    if current_state == 15:  # Goal state
        print("Reached goal!")
        break
    if current_state in [5, 7, 11, 12]:  # Hole states
        print("Fell in hole!")
        break

    action = int(optimal_policy[current_state])
    action_name = ['←', '↓', '→', '↑'][action]

    # Simple deterministic next state calculation (ignoring slippery ice)
    row, col = current_state // 4, current_state % 4
    if action == 0 and col > 0:  # Left
        next_state = current_state - 1
    elif action == 1 and row < 3:  # Down
        next_state = current_state + 4
    elif action == 2 and col < 3:  # Right
        next_state = current_state + 1
    elif action == 3 and row > 0:  # Up
        next_state = current_state - 4
    else:
        next_state = current_state  # Stay in place if move is invalid

    print(f"Step {step + 1}: State {current_state} → Action {action} ({action_name}) → State {next_state}")
    current_state = next_state
    path.append(current_state)

print(f"\nPath taken: {' → '.join(map(str, path))}")

print("\nPolicy iteration completed successfully!")
print("The algorithm found an optimal policy that maximizes expected rewards.")

Starting Policy Iteration...
Policy evaluation converged at iteration 1
Policy evaluation converged at iteration 37
Policy evaluation converged at iteration 111
Policy evaluation converged at iteration 116
Policy evaluation converged at iteration 155
Policy evaluation converged at iteration 159
Policy iteration converged at step 6

Optimal Value Function:
[[0.0688909  0.06141457 0.07440976 0.05580732]
 [0.09185454 0.         0.11220821 0.        ]
 [0.14543635 0.24749695 0.29961759 0.        ]
 [0.         0.3799359  0.63902015 0.        ]]

Optimal Policy:
[[0. 3. 0. 3.]
 [0. 0. 0. 0.]
 [3. 1. 0. 0.]
 [0. 2. 1. 0.]]

Optimal Policy (with direction symbols):
← ↑ ← ↑ 
← ← ← ← 
↑ ↓ ← ← 
← → ↓ ← 

Demonstrating the optimal policy with a few steps:
Starting at state: 0 (position [0,0])

Optimal actions from different states:
State 0 (position [0,0]): Action 0 (←)
State 1 (position [0,1]): Action 3 (↑)
State 2 (position [0,2]): Action 0 (←)
State 3 (position [0,3]): Action 3 (↑)
State 4 (po